# GenJam - Genetic Algorithm for Jazz Soloes

### Define Measure and Phrase Classes

In [1]:
import random

class Measure:
    events: int # Array of 8 consisting events
    fitness: int # The fitness of the measure
        
    def __init__(self, events=False):
        if events==False:
            self.events = [random.randint(0, 15) for _ in range(8)] # Randmoly allocate events
            self.fitness = 0
            pass
        
        else:
            self.events = events
            self.fitness = 0
            pass
    
    def evaluate(self, fitness):
        self.fitness = fitness
        pass
    
class Phrase:
    measures: int # Array of 4 consisting measures
    fitness: int # The fitness of the measure
        
    def __init__(self, measure_population, measures=False):
        if measures==False:
            self.measures = random.choices(measure_population, k=4) # Select 4 measures randomly allowed possible repetitions
            self.fitness = sum([self.measures[i].fitness for i in range(4)]) # Naive fitness alocation
            pass
        
        else:
            self.measures = measures
            self.fitness = 0
            pass
    
    def evaluate(self, fitness):
        self.fitness = fitness
        pass
    

### Define Measure and Phrase Crossovers

In [2]:
def measure_crossover(parent1, parent2):
    crossover_point = random.randint(1, 7)
    offspring1 = Measure(parent1.events[:crossover_point] + parent2.events[crossover_point:])
    offspring2 = Measure(parent2.events[:crossover_point] + parent1.events[crossover_point:])

    return offspring1, offspring2

def phrase_crossover(parent1, parent2):
    crossover_point = random.randint(1, 3)
    offspring1 = Phrase(parent1.measures[:crossover_point] + parent2.measures[crossover_point:])
    offspring2 = Phrase(parent2.measures[:crossover_point] + parent1.measures[crossover_point:])

    return offspring1, offspring2

### Define Measure Mutations

In [3]:
class Measure(Measure):
    
    def mu_transpose(self):
        scale = random.randint(-5, 5)
        self.events = [min(max((e+scale),1),14) if e!=0 and e!=15 else e for e in self.events]
        pass
    
    def mu_reverse(self):
        self.events.reverse()
        pass
    
    def mu_rotate(self):
        scale = random.randint(2, 5)
        self.events = self.events[scale:] + self.events[:scale]
        pass
    
    def mu_invert(self):
        self.events = [15-e for e in self.events]
        pass
    
    def mu_sortup(self):
        new_event_sorted = sorted([e for e in self.events if e!=0 and e!=15])
        self.events = [new_event_sorted.pop(0) if e!=0 and e!=15 else e for e in self.events]
        pass
    
    def mu_sortdown(self):
        new_event_sorted = sorted([e for e in self.events if e!=0 and e!=15], reverse=True)
        self.events = [new_event_sorted.pop(0) if e!=0 and e!=15 else e for e in self.events]
        pass
    
    def mu_invert_reverse(self):
        self.mu_invert()
        self.mu_reverse()
        pass

In [4]:
# Test measure mutation based on the example in the report
M = Measure([9, 7, 0, 5, 7, 15, 15, 0])
print("Original Measure: ", M.events)

M = Measure([9, 7, 0, 5, 7, 15, 15, 0])
M.mu_transpose()
print("Transpose (rand): ", M.events)

M = Measure([9, 7, 0, 5, 7, 15, 15, 0])
M.mu_reverse()
print("Reverse         : ", M.events)

M = Measure([9, 7, 0, 5, 7, 15, 15, 0])
M.mu_rotate()
print("Rotate (rand)   : ", M.events)

M = Measure([9, 7, 0, 5, 7, 15, 15, 0])
M.mu_invert()
print("Invert          : ", M.events)

M = Measure([9, 7, 0, 5, 7, 15, 15, 0])
M.mu_sortup()
print("Sort Up         : ", M.events)

M = Measure([9, 7, 0, 5, 7, 15, 15, 0])
M.mu_sortdown()
print("Sort Down       : ", M.events)

M = Measure([9, 7, 0, 5, 7, 15, 15, 0])
M.mu_invert_reverse()
print("Invert/Reverse  : ", M.events)

Original Measure:  [9, 7, 0, 5, 7, 15, 15, 0]
Transpose (rand):  [12, 10, 0, 8, 10, 15, 15, 0]
Reverse         :  [0, 15, 15, 7, 5, 0, 7, 9]
Rotate (rand)   :  [7, 15, 15, 0, 9, 7, 0, 5]
Invert          :  [6, 8, 15, 10, 8, 0, 0, 15]
Sort Up         :  [5, 7, 0, 7, 9, 15, 15, 0]
Sort Down       :  [9, 7, 0, 7, 5, 15, 15, 0]
Invert/Reverse  :  [15, 0, 0, 8, 10, 15, 8, 6]


### Define Phrase Mutations

In [5]:
import heapq

class Phrase(Phrase):
    
    def mu_reverse(self):
        self.measures.reverse()
        pass
    
    def mu_rotate(self):
        scale = random.randint(1, 3)
        self.measures = self.measures[scale:] + self.measures[:scale]
        pass
    
    def mu_sequence(self):
        index = random.randint(0, 3)  # Choose a replace index
        sequence = index + random.choice([-1, 1])  # Choose a sequence index
        if sequence==-1: sequence=1 # Only one choice for 0 and 3 indexes
        elif sequence==4: sequence=2
        self.measures[index] = self.measures[sequence] 
        pass
    
    def mu_repair(self, population):
        measure_population = population[0]
        random_measure = random.choices(measure_population)[0]
        least_fitness = min(range(4), key=lambda i: self.measures[i].fitness)
        self.measures[least_fitness] = random_measure
        pass
    
    def mu_super(self, population):
        measure_population = population[0]
        three_tournament = [random.choices(measure_population, k=3) for _ in range(4)]
        selected_super = [max(three_tournament[i], key=lambda x: x.fitness) for i in range(4)]
        random.shuffle(selected_super)
        self.measures = selected_super # Discard the child
        pass
    
    def mu_thinlick(self, population):
        measure_population = population[0]
        phrase_population = population[1]
        measure_counts = {}
        for measure in self.measures: # Set measures in this phrase
            if measure not in measure_counts:
                    measure_counts[measure] = 0
                    
        for phrase in phrase_population: # Count measures in population
            for measure in phrase.measures:
                if measure in measure_counts:
                    measure_counts[measure] += 1

        lick_index = self.measures.index(max(measure_counts, key=measure_counts.get)) # Obtain the first index the lick is appeared
        self.measures[lick_index] = random.choices(measure_population)[0]
        pass
    
    def mu_orphan(self, population):
        measure_population = population[0]
        phrase_population = population[1]
        measure_counts = {}
        for measure in measure_population: # Set measures in measure population
            if measure not in measure_counts:
                    measure_counts[measure] = 0
                    
        for phrase in phrase_population: # Count measures in population
            for measure in phrase.measures:
                if measure in measure_counts:
                    measure_counts[measure] += 1
                    
        self.measures = heapq.nsmallest(4, measure_counts, key=measure_counts.get) # Discard the child
        random.shuffle(self.measures) # Shuffle the measures randomly
                    
        

In [6]:
# Test phrase mutation based on the example in the report

def strp(phrase, measure_population):
    measure_index = [measure_population.index(m) for m in phrase.measures]
    return measure_index


# Create measure_population with 20 and phrase_population with 10 population
measure_population = [Measure() for _ in range(64)]
phrase_population = [Phrase(measure_population) for _ in range(10)] + [Phrase(measure_population, [measure_population[57], measure_population[57], measure_population[11], measure_population[38]])]



In [7]:
print("Original Measure: ", strp(phrase_population[10], measure_population))

phrase_population[10].mu_reverse()
print("Reverse         : ", strp(phrase_population[10], measure_population))

phrase_population[10].measures = [measure_population[57], measure_population[57], measure_population[11], measure_population[38]]
phrase_population[10].mu_rotate()
print("Rotate (rand)   : ", strp(phrase_population[10], measure_population))

phrase_population[10].measures = [measure_population[57], measure_population[57], measure_population[11], measure_population[38]]
phrase_population[10].mu_sequence()
print("Sequence Phrase : ", strp(phrase_population[10], measure_population))

phrase_population[10].measures = [measure_population[57], measure_population[57], measure_population[11], measure_population[38]]
phrase_population[10].mu_repair([measure_population])
print("Genetic Repair  : ", strp(phrase_population[10], measure_population))

phrase_population[10].measures = [measure_population[57], measure_population[57], measure_population[11], measure_population[38]]
phrase_population[10].mu_super([measure_population])
print("Super Phrase    : ", strp(phrase_population[10], measure_population))

phrase_population[10].measures = [measure_population[57], measure_population[57], measure_population[11], measure_population[38]]
phrase_population[10].mu_thinlick([measure_population, phrase_population])
print("Lick Thinner    : ", strp(phrase_population[10], measure_population))

phrase_population[10].measures = [measure_population[57], measure_population[57], measure_population[11], measure_population[38]]
phrase_population[10].mu_orphan([measure_population, phrase_population])
print("Orphan Phrase   : ", strp(phrase_population[10], measure_population))

Original Measure:  [57, 57, 11, 38]
Reverse         :  [38, 11, 57, 57]
Rotate (rand)   :  [38, 57, 57, 11]
Sequence Phrase :  [57, 57, 11, 38]
Genetic Repair  :  [3, 57, 11, 38]
Super Phrase    :  [23, 45, 39, 31]
Lick Thinner    :  [3, 57, 11, 38]
Orphan Phrase   :  [8, 5, 9, 4]


### Add random mutation function to Measure and Phrase

In [8]:
class Measure(Measure):
    def __init__(self, events=False):
        self.mutations = [
                self.mu_transpose,
                self.mu_reverse,
                self.mu_rotate,
                self.mu_invert,
                self.mu_sortup,
                self.mu_sortdown,
                self.mu_invert_reverse
            ]
        if events==False:
            self.events = [random.randint(0, 15) for _ in range(8)] # Randmoly allocate events
            self.fitness = 0
            pass
        
        else:
            self.events = events
            self.fitness = 0
            pass
        
    def apply_random_mutation(self):
        random_mutation = random.choice(self.mutations)
        random_mutation()
        pass
    
class Phrase(Phrase): 
    def __init__(self, measure_population, measures=False):
        self.mutations = [
                self.mu_reverse,
                self.mu_rotate,
                self.mu_sequence,
                self.mu_repair,
                self.mu_super,
                self.mu_thinlick,
                self.mu_orphan
            ]
        if measures==False:
            self.measures = random.choices(measure_population, k=4) # Select 4 measures randomly allowed possible repetitions
            self.fitness = sum([self.measures[i].fitness for i in range(4)]) # Naive fitness alocation
            pass
        
        else:
            self.measures = measures
            self.fitness = 0
            pass

    def apply_random_mutation(self, phrase_population, measure_population):
        random_mutation = random.choice(self.mutations)
        
        # Apply the random function with different input sizes
        if random_mutation.__code__.co_argcount > 1: random_mutation([measure_population, phrase_population])
        else: random_mutation()
        pass

### Define GenJam

In [9]:
class GenJam:
    def __init__(self, num_measures = 64, num_phrases = 48, num_measure_tournament = 4, num_phrase_tournament = 4):
        self.num_measures = num_measures
        self.num_phrases = num_phrases
        self.num_measure_tournament = num_measure_tournament
        self.num_phrase_tournament = num_phrase_tournament
        self.measure_population = [Measure() for _ in range(num_measures)]
        self.phrase_population = [Phrase(measure_population) for _ in range(num_phrases)]
        pass
    
    def generate(self, num_generations = 1):
        for iteration in range(num_generations):
            self.select_generate_measure()
            self.select_generate_phrase()
            print("An Iteration is Passed. The Best Achived Phrase: ")
            self.phrase_population.sort(key=lambda x: x.fitness)
            self.print_phrase(phrase_population[0])            
        pass
    
    def select_generate_measure(self):
        for i in range(self.num_measures//4):
            
            # Selection process
            p1, p2 = self.tournament_selection(self.measure_population, self.num_measure_tournament)
            
            # Crossover process
            o1, o2 = measure_crossover(p1, p2)
            
            # Mutation process
            random.choice([o1, o2]).apply_random_mutation()
            
            # Evaluation Process
            o1.fitness = self.print_measure_fit(o1)
            o2.fitness = self.print_measure_fit(o2)
            
            # Replace peocess
            self.measure_population.sort(key=lambda x: x.fitness)
            self.measure_population[:2] = [o1, o2]            
        pass
    
    def select_generate_phrase(self):
        for i in range(self.num_phrases//4):
            
            # Selection process
            p1, p2 = self.tournament_selection(self.phrase_population, self.num_phrase_tournament)
            
            # Crossover process
            o1, o2 = phrase_crossover(p1, p2)
            
            # Mutation process
            random.choice([o1, o2]).apply_random_mutation(self.phrase_population, self.measure_population)
            
            # Evaluation Process
            o1.fitness = self.print_phrase_fit(o1)
            o2.fitness = self.print_phrase_fit(o2)
            
            # Replace peocess
            self.phrase_population.sort(key=lambda x: x.fitness)
            self.phrase_population[:2] = [o1, o2]            
        pass
    
    def tournament_selection(self, population, k):
        selection = random.sample(population, k)
        parents = sorted(selection, key=lambda x: x.fitness, reverse=True)[:2]
        return parents[0], parents[1]
    
    def print_measure_fit(self, m):
        return int(input(str(m.events)+", Evaluated Fitness for this Measure: "))
        
    def print_phrase_fit(self, p):
        measures_events = [m.events for m in p.measures]
        return int(input(str(measures_events)+", Evaluated Fitness for this Phrase: "))
    
    def print_phrase(self, p):
        measures_events = [m.events for m in p.measures]
        print(measures_events)
        pass

In [10]:
jamer = GenJam()

In [11]:
jamer.generate()

[14, 9, 15, 4, 15, 3, 0, 11], Evaluated Fitness for this Measure: 1
[15, 3, 14, 13, 8, 6, 2, 13], Evaluated Fitness for this Measure: 2
[8, 1, 2, 11, 11, 0, 6, 1], Evaluated Fitness for this Measure: 3
[7, 10, 0, 5, 15, 3, 0, 11], Evaluated Fitness for this Measure: 4
[14, 9, 10, 13, 13, 5, 7, 4], Evaluated Fitness for this Measure: 5
[11, 0, 3, 15, 4, 15, 0, 1], Evaluated Fitness for this Measure: 6
[9, 13, 5, 0, 2, 6, 9, 1], Evaluated Fitness for this Measure: 7
[6, 12, 11, 6, 4, 2, 12, 9], Evaluated Fitness for this Measure: 8
[5, 8, 10, 3, 5, 6, 5, 9], Evaluated Fitness for this Measure: 9
[8, 12, 11, 9, 0, 7, 8, 0], Evaluated Fitness for this Measure: 0
[13, 13, 0, 10, 7, 5, 4, 3], Evaluated Fitness for this Measure: 1
[7, 5, 4, 5, 9, 13, 14, 7], Evaluated Fitness for this Measure: 2
[14, 15, 0, 8, 1, 14, 2, 12], Evaluated Fitness for this Measure: 3
[8, 0, 3, 10, 13, 8, 8, 9], Evaluated Fitness for this Measure: 4
[8, 0, 3, 10, 13, 8, 7, 0], Evaluated Fitness for this Measure: 5
